In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0) 

import pandas as pd
import random
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

import os

import numpy as np
import torch.nn.functional as F

from google.colab import drive
drive.mount('/content/drive')

from sklearn.metrics import accuracy_score
from datetime import datetime
from IPython.display import clear_output 

Mounted at /content/drive


In [2]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

1.12.1+cu113
     |████████████████████████████████| 7.9 MB 3.9 MB/s 
     |████████████████████████████████| 3.5 MB 6.9 MB/s 
     |████████████████████████████████| 2.4 MB 7.2 MB/s 


In [3]:
train_dataset=[]
graphlist=[]
import networkx as nx
import torch_geometric
from torch_geometric.data import Data, Dataset
from torch_geometric.loader import DataLoader
#from torch.utils.data import DataLoader
from torch_geometric.utils import degree
from torch_geometric.utils import erdos_renyi_graph, to_networkx, from_networkx


%cd /content/drive/MyDrive/ADNI2/Controls
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp)  
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = 1
      graphlist.append(data)
      #graphlist.append([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])

%cd /content/drive/MyDrive/ADNI2/Patients
for i in range(1, 110):
    if os.path.isfile('{0:0>3}'.format(i)+'.txt'):
      tmp=pd.read_csv('{0:0>3}'.format(i)+'.txt', header=None)
      tmp=(tmp.to_numpy())
      tmp=torch.from_numpy(tmp) 
      tmp[tmp<=0.4]=0
      #print([tmp.type(torch.FloatTensor),torch.ones(1).type(torch.long)])
      G = nx.from_numpy_matrix(np.array(tmp))
      strength = G.degree(weight='weight')
      x = []
      for i in strength:
        x.extend([i[1]])
      x = np.array(x).reshape(90,1)
      data = from_networkx(G)
      #x = torch.tensor([[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1],[1] ], dtype=torch.float)
      data.x = torch.FloatTensor(x)
      data.y = 0
      graphlist.append(data)
      #graphlist.append([data, torch.zeros(1).type(torch.long)])

      #graphlist.append([tmp.type(torch.FloatTensor),torch.zeros(1).type(torch.long)])

/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Controls
/content/drive/.shortcut-targets-by-id/1n9P-S3rsi_g9yeb5KW1h7v-Bte7Cy43G/ADNI2/Patients


In [4]:
graphlist[0]

Data(edge_index=[2, 3054], weight=[3054], num_nodes=90, x=[90, 1], y=1)

In [5]:
#from torch.utils.data.dataset import random_split
dataset = random.sample(graphlist, len(graphlist))
train_dataset = dataset[:150]
test_dataset = dataset[150:]
train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=25, shuffle=True)

In [6]:
from torch_geometric.nn import GATConv, GraphConv
from torch_geometric.nn.conv.gatv2_conv import GATv2Conv
from torch_geometric.nn import global_mean_pool, global_sort_pool

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()

        self.in_head = 8
        self.out_head = 1
        self.conv1 = GraphConv(in_channels=1, out_channels=16)
        self.conv2 = GraphConv(in_channels=16, out_channels=32)
        self.conv3 = GraphConv(in_channels=32, out_channels=32)
        self.lin1 = torch.nn.Linear(32, 16)
        self.lin2 = torch.nn.Linear(16, 1)
        self.sig = torch.nn.Sigmoid()

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch       
        x = self.conv1(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv2(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = self.conv3(x, edge_index, weight)
        x = F.relu(x)
        #print(x.size())
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        #x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin1(x)
        x = self.lin2(x)
        #print(x)
        
        return x
        #return F.log_softmax(x, dim=1)

In [7]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 55.5650
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 408.2015
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 52.1766
Epoch: 004, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 18.8727
Epoch: 005, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 12.8663
Epoch: 006, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 16.9010
Epoch: 007, Train Acc: 0.8200, Test Acc: 0.7353, Loss: 4.2540
Epoch: 008, Train Acc: 0.6933, Test Acc: 0.6471, Loss: 4.0700
Epoch: 009, Train Acc: 0.6000, Test Acc: 0.6471, Loss: 17.3286
Epoch: 010, Train Acc: 0.4933, Test Acc: 0.5294, Loss: 7.8483
Epoch: 011, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 8.0658
Epoch: 012, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 21.0807
Epoch: 013, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 30.8990
Epoch: 014, Train Acc: 0.5133, Test Acc: 0.5441, Loss: 2.3681
Epoch: 015, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 4.5947
Epoch: 016, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 9.753

In [8]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GCNConv, JumpingKnowledge, global_mean_pool


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.num_layers  = 3
        self.hidden = 16
        self.conv1 = GCNConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(GCNConv(self.hidden, self.hidden))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index,weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index, weight))
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [9]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5000, Test Acc: 0.5294, Loss: 0.6768
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6475
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5441, Loss: 0.8191
Epoch: 004, Train Acc: 0.7667, Test Acc: 0.6765, Loss: 0.6588
Epoch: 005, Train Acc: 0.7800, Test Acc: 0.7353, Loss: 0.6703
Epoch: 006, Train Acc: 0.4867, Test Acc: 0.5441, Loss: 0.6439
Epoch: 007, Train Acc: 0.8867, Test Acc: 0.8529, Loss: 0.6322
Epoch: 008, Train Acc: 0.9067, Test Acc: 0.8676, Loss: 0.6338
Epoch: 009, Train Acc: 0.8867, Test Acc: 0.8382, Loss: 0.6216
Epoch: 010, Train Acc: 0.6933, Test Acc: 0.6618, Loss: 0.5933
Epoch: 011, Train Acc: 0.5733, Test Acc: 0.6176, Loss: 0.6311
Epoch: 012, Train Acc: 0.8800, Test Acc: 0.8382, Loss: 0.6166
Epoch: 013, Train Acc: 0.8800, Test Acc: 0.8529, Loss: 0.4826
Epoch: 014, Train Acc: 0.8267, Test Acc: 0.7941, Loss: 0.5754
Epoch: 015, Train Acc: 0.7400, Test Acc: 0.7206, Loss: 0.3749
Epoch: 016, Train Acc: 0.9000, Test Acc: 0.8676, Loss: 0.4169
Epoch: 0

In [10]:
from torch.nn import Linear
from torch_geometric.nn import GINConv, JumpingKnowledge, global_mean_pool
from torch.nn import BatchNorm1d as BN
from torch.nn import Linear, ReLU, Sequential
#from torch_geometric.nn import JumpingKnowledge, SAGEConv, global_mean_pool

class GIN(torch.nn.Module):
    def __init__(self):
        super(GIN, self).__init__()
        self.conv1 = GINConv(
            Sequential(
                Linear(1, 16),
                ReLU(),
                Linear(16, 16),
                ReLU(),
                BN(16),
            ), train_eps=False)
        self.conv2 = GINConv(
            Sequential(
                Linear(16, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.conv3 = GINConv(
            Sequential(
                Linear(32, 32),
                ReLU(),
                Linear(32, 32),
                ReLU(),
                BN(32),
            ), train_eps=False)
        self.lin1 = Linear(32, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = global_mean_pool(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [11]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GIN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.5411
Epoch: 002, Train Acc: 0.5267, Test Acc: 0.5441, Loss: 0.4140
Epoch: 003, Train Acc: 0.6667, Test Acc: 0.6471, Loss: 0.6245
Epoch: 004, Train Acc: 0.7133, Test Acc: 0.6618, Loss: 0.5385
Epoch: 005, Train Acc: 0.7933, Test Acc: 0.7206, Loss: 0.5399
Epoch: 006, Train Acc: 0.8200, Test Acc: 0.7353, Loss: 0.3186
Epoch: 007, Train Acc: 0.8733, Test Acc: 0.7941, Loss: 0.1793
Epoch: 008, Train Acc: 0.8800, Test Acc: 0.8235, Loss: 0.2547
Epoch: 009, Train Acc: 0.8867, Test Acc: 0.8235, Loss: 0.5389
Epoch: 010, Train Acc: 0.8867, Test Acc: 0.8382, Loss: 0.2096
Epoch: 011, Train Acc: 0.8733, Test Acc: 0.8235, Loss: 0.4088
Epoch: 012, Train Acc: 0.8800, Test Acc: 0.8382, Loss: 0.2642
Epoch: 013, Train Acc: 0.8733, Test Acc: 0.7941, Loss: 0.2362
Epoch: 014, Train Acc: 0.8733, Test Acc: 0.8088, Loss: 0.3064
Epoch: 015, Train Acc: 0.8467, Test Acc: 0.7794, Loss: 0.2689
Epoch: 016, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.2934
Epoch: 0

In [12]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.data import Batch
from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
    graclus,
    max_pool,
)

num_layers  = 4
hidden = 16
class Graclus(torch.nn.Module):
    def __init__(self):
        super(Graclus, self).__init__()
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        for i in range(num_layers - 1):
            self.convs.append(GraphConv(16, 16, aggr='mean'))
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(64, 16)
        self.lin2 = Linear(16, 1)

    def forward(self, data):
        x, edge_index, weight , batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                cluster = graclus(edge_index,num_nodes=x.size(0))
                data = Batch(x=x, edge_index=edge_index, batch=batch)
                data = max_pool(cluster, data)
                x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        x = self.lin2(x)
        return x

In [13]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Graclus().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6874
Epoch: 002, Train Acc: 0.8600, Test Acc: 0.8529, Loss: 0.6894
Epoch: 003, Train Acc: 0.5867, Test Acc: 0.6176, Loss: 0.6273
Epoch: 004, Train Acc: 0.8267, Test Acc: 0.7500, Loss: 0.6088
Epoch: 005, Train Acc: 0.7533, Test Acc: 0.6324, Loss: 0.6042
Epoch: 006, Train Acc: 0.8200, Test Acc: 0.6618, Loss: 0.4638
Epoch: 007, Train Acc: 0.8533, Test Acc: 0.8088, Loss: 0.5131
Epoch: 008, Train Acc: 0.8733, Test Acc: 0.8529, Loss: 0.3484
Epoch: 009, Train Acc: 0.9133, Test Acc: 0.9118, Loss: 0.5238
Epoch: 010, Train Acc: 0.8800, Test Acc: 0.8382, Loss: 0.4751
Epoch: 011, Train Acc: 0.8267, Test Acc: 0.7353, Loss: 0.4479
Epoch: 012, Train Acc: 0.9333, Test Acc: 0.8971, Loss: 0.4902
Epoch: 013, Train Acc: 0.8533, Test Acc: 0.7500, Loss: 0.1143
Epoch: 014, Train Acc: 0.8867, Test Acc: 0.9412, Loss: 0.3052
Epoch: 015, Train Acc: 0.9267, Test Acc: 0.9265, Loss: 0.2119
Epoch: 016, Train Acc: 0.9400, Test Acc: 0.8676, Loss: 0.3737
Epoch: 0

In [14]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    TopKPooling,
    global_mean_pool,
)


class TopK(torch.nn.Module):
    def __init__(self):
        super(TopK, self).__init__()
        self.ratio = 0.8
        self.num_layers = 3
        self.hidden = 16
        self.conv1 = GraphConv(1, 16, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(16, 16, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [TopKPooling(16, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(48, 16)
        self.lin2 = Linear(16, 1)


    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        #return F.log_softmax(x, dim=-1)
        return x

In [15]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TopK().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6723
Epoch: 002, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.6818
Epoch: 003, Train Acc: 0.8600, Test Acc: 0.8529, Loss: 0.6839
Epoch: 004, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6944
Epoch: 005, Train Acc: 0.4933, Test Acc: 0.5441, Loss: 0.6906
Epoch: 006, Train Acc: 0.5200, Test Acc: 0.5588, Loss: 0.6742
Epoch: 007, Train Acc: 0.4867, Test Acc: 0.5441, Loss: 0.6761
Epoch: 008, Train Acc: 0.5867, Test Acc: 0.6029, Loss: 0.6521
Epoch: 009, Train Acc: 0.6400, Test Acc: 0.6324, Loss: 0.6649
Epoch: 010, Train Acc: 0.6467, Test Acc: 0.6324, Loss: 0.6381
Epoch: 011, Train Acc: 0.8933, Test Acc: 0.8824, Loss: 0.6469
Epoch: 012, Train Acc: 0.7400, Test Acc: 0.6912, Loss: 0.5676
Epoch: 013, Train Acc: 0.9000, Test Acc: 0.8824, Loss: 0.5277
Epoch: 014, Train Acc: 0.8800, Test Acc: 0.8824, Loss: 0.5224
Epoch: 015, Train Acc: 0.5933, Test Acc: 0.5000, Loss: 0.4164
Epoch: 016, Train Acc: 0.6733, Test Acc: 0.5882, Loss: 0.3700
Epoch: 0

In [16]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    GraphConv,
    JumpingKnowledge,
    SAGPooling,
    global_mean_pool,
)


class SAGPool(torch.nn.Module):
    def __init__(self):
        super(SAGPool, self).__init__()
        self.num_layers = 4
        self.hidden = 32
        self.ratio = 0.8
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [SAGPooling(self.hidden, self.ratio) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight, batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, _, batch, _, _ = pool(x, edge_index,
                                                     batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)

In [17]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SAGPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5133, Test Acc: 0.4559, Loss: 0.6756
Epoch: 002, Train Acc: 0.3733, Test Acc: 0.4118, Loss: 0.7384
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6917
Epoch: 004, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6576
Epoch: 005, Train Acc: 0.6467, Test Acc: 0.6765, Loss: 0.6847
Epoch: 006, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.7308
Epoch: 007, Train Acc: 0.8933, Test Acc: 0.9265, Loss: 0.6575
Epoch: 008, Train Acc: 0.8067, Test Acc: 0.7794, Loss: 0.6173
Epoch: 009, Train Acc: 0.6000, Test Acc: 0.6324, Loss: 0.5565
Epoch: 010, Train Acc: 0.9067, Test Acc: 0.9265, Loss: 0.4622
Epoch: 011, Train Acc: 0.8000, Test Acc: 0.7794, Loss: 1.2043
Epoch: 012, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.5516
Epoch: 013, Train Acc: 0.7933, Test Acc: 0.7647, Loss: 0.5139
Epoch: 014, Train Acc: 0.9000, Test Acc: 0.9265, Loss: 0.4103
Epoch: 015, Train Acc: 0.9333, Test Acc: 0.9118, Loss: 0.2296
Epoch: 016, Train Acc: 0.9067, Test Acc: 0.9265, Loss: 0.1894
Epoch: 0

In [18]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    EdgePooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class EdgePool(torch.nn.Module):
    def __init__(self):
        super(EdgePool, self).__init__()
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend(
            [EdgePooling(self.hidden) for i in range((self.num_layers) // 2)])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, weight ,  batch = data.x, data.edge_index, data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = F.relu(conv(x, edge_index))
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, batch, _ = pool(x, edge_index, batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [19]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = EdgePool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.8653
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6780
Epoch: 003, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.6845
Epoch: 004, Train Acc: 0.6000, Test Acc: 0.6029, Loss: 0.6622
Epoch: 005, Train Acc: 0.9000, Test Acc: 0.9118, Loss: 0.6183
Epoch: 006, Train Acc: 0.8400, Test Acc: 0.8088, Loss: 0.4492
Epoch: 007, Train Acc: 0.9267, Test Acc: 0.8529, Loss: 0.4750
Epoch: 008, Train Acc: 0.9133, Test Acc: 0.8971, Loss: 0.2329
Epoch: 009, Train Acc: 0.7200, Test Acc: 0.7206, Loss: 0.2743
Epoch: 010, Train Acc: 0.6600, Test Acc: 0.5588, Loss: 0.6183
Epoch: 011, Train Acc: 0.8200, Test Acc: 0.7941, Loss: 0.5084
Epoch: 012, Train Acc: 0.8867, Test Acc: 0.7794, Loss: 0.2266
Epoch: 013, Train Acc: 0.9067, Test Acc: 0.8971, Loss: 0.2833
Epoch: 014, Train Acc: 0.9400, Test Acc: 0.8971, Loss: 0.2580
Epoch: 015, Train Acc: 0.9400, Test Acc: 0.9265, Loss: 0.4293
Epoch: 016, Train Acc: 0.9467, Test Acc: 0.9265, Loss: 0.1485
Epoch: 0

In [20]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import GlobalAttention, SAGEConv


class GlobalAttentionNet(torch.nn.Module):
    def __init__(self):
        super(GlobalAttentionNet, self).__init__()
        self.num_layers = 3
        self.hidden  = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.att = GlobalAttention(Linear(self.hidden, 1))
        self.lin1 = Linear(self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.att(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [21]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GlobalAttentionNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5333, Test Acc: 0.5735, Loss: 0.6865


/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.GlobalAttention' is deprecated, use 'nn.aggr.AttentionalAggregation' instead
  warnings.warn(out)


Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6521
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6829
Epoch: 004, Train Acc: 0.8467, Test Acc: 0.8529, Loss: 0.6293
Epoch: 005, Train Acc: 0.7933, Test Acc: 0.8235, Loss: 0.5259
Epoch: 006, Train Acc: 0.7133, Test Acc: 0.7647, Loss: 0.6220
Epoch: 007, Train Acc: 0.8400, Test Acc: 0.8235, Loss: 0.5353
Epoch: 008, Train Acc: 0.8267, Test Acc: 0.8529, Loss: 0.4477
Epoch: 009, Train Acc: 0.8400, Test Acc: 0.8676, Loss: 0.3626
Epoch: 010, Train Acc: 0.8600, Test Acc: 0.8676, Loss: 0.4382
Epoch: 011, Train Acc: 0.8667, Test Acc: 0.8676, Loss: 0.2097
Epoch: 012, Train Acc: 0.8733, Test Acc: 0.8676, Loss: 0.2886
Epoch: 013, Train Acc: 0.8400, Test Acc: 0.8529, Loss: 0.2976
Epoch: 014, Train Acc: 0.8667, Test Acc: 0.8676, Loss: 0.5203
Epoch: 015, Train Acc: 0.8667, Test Acc: 0.8676, Loss: 0.2253
Epoch: 016, Train Acc: 0.8667, Test Acc: 0.8676, Loss: 0.3937
Epoch: 017, Train Acc: 0.8800, Test Acc: 0.8971, Loss: 0.3644
Epoch: 0

In [22]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import SAGEConv, Set2Set


class Set2SetNet(torch.nn.Module):
    def __init__(self):
        super(Set2SetNet,self).__init__()
        self.hidden = 16
        self.num_layers = 3 
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.set2set = Set2Set(self.hidden, processing_steps=4)
        self.lin1 = Linear(2 * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = self.set2set(x, batch)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [23]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Set2SetNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 300):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.Set2Set' is deprecated, use 'nn.aggr.Set2Set' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.6267, Test Acc: 0.5294, Loss: 0.6864
Epoch: 002, Train Acc: 0.8733, Test Acc: 0.8382, Loss: 0.6795
Epoch: 003, Train Acc: 0.8733, Test Acc: 0.8235, Loss: 0.6806
Epoch: 004, Train Acc: 0.8133, Test Acc: 0.7647, Loss: 0.6806
Epoch: 005, Train Acc: 0.7467, Test Acc: 0.7206, Loss: 0.6776
Epoch: 006, Train Acc: 0.6933, Test Acc: 0.6618, Loss: 0.6874
Epoch: 007, Train Acc: 0.6667, Test Acc: 0.6471, Loss: 0.6784
Epoch: 008, Train Acc: 0.6533, Test Acc: 0.6471, Loss: 0.6823
Epoch: 009, Train Acc: 0.6467, Test Acc: 0.6471, Loss: 0.6647
Epoch: 010, Train Acc: 0.6200, Test Acc: 0.6471, Loss: 0.6753
Epoch: 011, Train Acc: 0.6400, Test Acc: 0.6471, Loss: 0.6634
Epoch: 012, Train Acc: 0.6467, Test Acc: 0.6471, Loss: 0.6660
Epoch: 013, Train Acc: 0.6400, Test Acc: 0.6471, Loss: 0.6638
Epoch: 014, Train Acc: 0.6267, Test Acc: 0.6471, Loss: 0.6772
Epoch: 015, Train Acc: 0.6533, Test Acc: 0.6471, Loss: 0.6714
Epoch: 016, Train Acc: 0.6267, Test Acc: 0.6471, Loss: 0.6837
Epoch: 0

In [24]:
import torch
import torch.nn.functional as F
from torch.nn import Conv1d, Linear

from torch_geometric.nn import SAGEConv, global_sort_pool


class SortPool(torch.nn.Module):
    def __init__(self):
        super(SortPool, self).__init__()
        self.k = 30
        self.num_layers = 3 
        self.hidden = 16
        self.conv1 = SAGEConv(1, self.hidden)
        self.convs = torch.nn.ModuleList()
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(self.hidden, self.hidden))
        self.conv1d = Conv1d(self.hidden, 32, 5)
        self.lin1 = Linear(32 * (self.k - 5 + 1), self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = F.relu(self.conv1(x, edge_index))
        for conv in self.convs:
            x = F.relu(conv(x, edge_index))
        x = global_sort_pool(x, batch, self.k)
        x = x.view(len(x), self.k, -1).permute(0, 2, 1)
        x = F.relu(self.conv1d(x))
        x = x.view(len(x), -1)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x

In [25]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SortPool().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 200):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

/usr/local/lib/python3.7/dist-packages/torch_geometric/deprecation.py:12: UserWarning: 'nn.glob.global_sort_pool' is deprecated, use 'nn.aggr.SortAggr' instead
  warnings.warn(out)


Epoch: 001, Train Acc: 0.4800, Test Acc: 0.4706, Loss: 0.6939
Epoch: 002, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6825
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.7065
Epoch: 004, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6884
Epoch: 005, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6993
Epoch: 006, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6875
Epoch: 007, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6871
Epoch: 008, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6874
Epoch: 009, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6985
Epoch: 010, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6888
Epoch: 011, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6862
Epoch: 012, Train Acc: 0.4933, Test Acc: 0.5294, Loss: 0.6749
Epoch: 013, Train Acc: 0.5133, Test Acc: 0.5294, Loss: 0.6798
Epoch: 014, Train Acc: 0.4933, Test Acc: 0.5294, Loss: 0.6954
Epoch: 015, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.6889
Epoch: 016, Train Acc: 0.6000, Test Acc: 0.6176, Loss: 0.6754
Epoch: 0

In [26]:
import torch
import torch.nn.functional as F
from torch.nn import Linear

from torch_geometric.nn import (
    ASAPooling,
    GraphConv,
    JumpingKnowledge,
    global_mean_pool,
)


class ASAP(torch.nn.Module):
    def __init__(self):
        super(ASAP, self).__init__()
        self.num_layers = 5 
        self.hidden  = 32
        self.ratio = 0.8
        self.dropout = 0.2
        self.conv1 = GraphConv(1, self.hidden, aggr='mean')
        self.convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.convs.extend([
            GraphConv(self.hidden, self.hidden, aggr='mean')
            for i in range(self.num_layers - 1)
        ])
        self.pools.extend([
            ASAPooling(self.hidden, self.ratio, dropout=self.dropout)
            for i in range((self.num_layers) // 2)
        ])
        self.jump = JumpingKnowledge(mode='cat')
        self.lin1 = Linear(self.num_layers * self.hidden, self.hidden)
        self.lin2 = Linear(self.hidden, 1)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index,data.weight, data.batch
        x = F.relu(self.conv1(x, edge_index, edge_weight))
        xs = [global_mean_pool(x, batch)]
        for i, conv in enumerate(self.convs):
            x = conv(x=x, edge_index=edge_index, edge_weight=edge_weight)
            x = F.relu(x)
            xs += [global_mean_pool(x, batch)]
            if i % 2 == 0 and i < len(self.convs) - 1:
                pool = self.pools[i // 2]
                x, edge_index, edge_weight, batch, _ = pool(
                    x=x, edge_index=edge_index, edge_weight=edge_weight,
                    batch=batch)
        x = self.jump(xs)
        x = F.relu(self.lin1(x))
        #x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin2(x)
        return x
        #return F.log_softmax(x, dim=-1)


In [27]:
from IPython.display import Javascript
display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
sig=torch.nn.Sigmoid()
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ASAP().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
correct = 0

def test(loader):
     model.eval()

     correct = 0
     for data in loader:  # Iterate in batches over the training/test dataset.

         out = model(data.to(device))  
         pred = sig(out)>0.5  # Use the class with highest probability.
         #print(out.size())
         #print(int((pred == data.y).sum()))
         correct += int((pred.view(-1) == data.y).sum())
         #print(correct)  # Check against ground-truth labels.
     return correct / len(loader.dataset)  # Derive ratio of correct predictions.


for epoch in range(1, 100):
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         out = model(data.to(device))
         out = out.view(-1)
         loss = criterion(out, data.y.float())  # Compute the loss.
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}, Loss: {loss:.4f}')

<IPython.core.display.Javascript object>

Epoch: 001, Train Acc: 0.5133, Test Acc: 0.4706, Loss: 0.7247
Epoch: 002, Train Acc: 0.5733, Test Acc: 0.5000, Loss: 0.6389
Epoch: 003, Train Acc: 0.4867, Test Acc: 0.5294, Loss: 0.5311
Epoch: 004, Train Acc: 0.5800, Test Acc: 0.5000, Loss: 0.7007
Epoch: 005, Train Acc: 0.9000, Test Acc: 0.9118, Loss: 0.4907
Epoch: 006, Train Acc: 0.8333, Test Acc: 0.7794, Loss: 0.4515
Epoch: 007, Train Acc: 0.8267, Test Acc: 0.7794, Loss: 0.4001
Epoch: 008, Train Acc: 0.9133, Test Acc: 0.8676, Loss: 0.4318
Epoch: 009, Train Acc: 0.6867, Test Acc: 0.5735, Loss: 0.2289
Epoch: 010, Train Acc: 0.9133, Test Acc: 0.8824, Loss: 0.3024
Epoch: 011, Train Acc: 0.9000, Test Acc: 0.8824, Loss: 0.2542
Epoch: 012, Train Acc: 0.9067, Test Acc: 0.9118, Loss: 0.2040
Epoch: 013, Train Acc: 0.9133, Test Acc: 0.8824, Loss: 0.2939
Epoch: 014, Train Acc: 0.9133, Test Acc: 0.8824, Loss: 0.1553
Epoch: 015, Train Acc: 0.9067, Test Acc: 0.9118, Loss: 0.2793
Epoch: 016, Train Acc: 0.9067, Test Acc: 0.9118, Loss: 0.1258
Epoch: 0